In [ ]:
'''imports'''

import numpy as np
import math
import cvxpy as cp
import numbers
from cvxpy.expressions.expression import Expression
from IPython.display import clear_output
from scipy.special import comb
from scipy.spatial import ConvexHull
import time
%matplotlib inline
%config InlineBackend.figure_format='retina'
clear_output()

In [ ]:
TOLFEAS = 1e-5
PENFEAS = 1e5
TOLPRUNE = 1e-5

class Problem:
    """
    A class used to represent a problem of
    minimizing a convex function F over a convex domain X under a probabilistic constraint.
    
                    min_x in X F(x)  s.t. P_xi[c(x,xi) <= 0] >= varsigma_prob
                    
    with support(xi) = {xi_1,...,xi_N} and P[xi = xi_s] = pi_s for every s in [N]
    
    Hypothesis:: 
            x is in R^d
            xi is in R^p for p = 2 or 3
                    
    Attributes
    ----------
    objective : Expression
        A convex cvxpy expression
    constraints : list
        A list of cvxpy constraints, i.e. X
    scenarios : list
        List of N numpy ndarrays, xi_s, each of which of size p = 2 or 3 
    cstr_gen : callable
        The call cstr_gen(xi) for any numpy ndarray of size p (i.e. scenarios' dimension) returns the cvxpy Expression c(x,xi) 
    safe_indices : list
        List of indices in range(N) linked to constraints that must be satisfied at an 
    varsigma_prob : float
        Minimal level of confidence for the probabilistic constraint
    c_bound : list
        c_bound is a list of cvxpy xi-parametric concave objective to maximize on the current X_S(S=safe_indices); each problem comes with an optimal 
        x_bound(xi) at which one needs to evaluate the constraint cstr_gen(xi) => the bound should become
    vars_ : list
        The cvxpy Variables involved in the problem
    ....
        
    dimensions:: p and N
    """

    def __init__(self, objective, constraints=[],tau=1,scenarios=[],pis=None,cstr_gen=lambda xi:0.0,\
                 c_bound=None,rel_tol=1e-2,used_solver=cp.MOSEK,individual_nus_comp=None):
        assert isinstance(objective,Expression) and objective.is_dcp(),'objective should be a DCP cvxpy Expression'
        if objective.is_convex():
            self.objective = objective
        else:
            self.objective = - objective 
        for cstr in constraints:
            assert cstr.is_dcp(),'constraints must be CONVEX'
        self.constraints = constraints
        self.vars_ = list(set(self.objective.variables()))
        for expr in self.constraints:
            self.vars_ += expr.variables()
        self.slack_feas = cp.Variable(1,nonneg=True)
        self.objective += PENFEAS*self.slack_feas[0]
        self.vars_ += [self.slack_feas]
        self.vars_ = list(set(self.vars_))
        assert isinstance(scenarios,list) or isinstance(scenarios,np.ndarray), 'scenarios should be a list of N numpy ndarrays of size p'
        self.N = len(scenarios)
        for s,scenario in enumerate(scenarios):
            if s==0:
                assert isinstance(scenario,np.ndarray),'type mistmatch... check input'
                self.p = len(scenario)
            else:
                assert isinstance(scenario,np.ndarray) and len(scenario)==self.p,'dimension/type mistmatch... check input'
        if isinstance(scenarios,list):
            self.scenarios = np.array(scenarios)
        else:
            self.scenarios = scenarios
        assert isinstance(tau,float),'tau should be a float in (0,1)'
        self.varsigma_prob = max(min(1,1-tau),0)
        self.safe_indices,self.pruned_indices,self.selectable = [],[],[]
        self._refresh()
        self.cuts = []
        if pis is None:
            self.pis = np.ones(self.N)/self.N
        else:
            self.pis = pis.copy()
        self.LB = -np.inf
        self.es_lb = None
        self.UB = np.inf
        self.es_ub = None
        assert isinstance(rel_tol,float) and rel_tol>=0,'rel_tol should be a nonnegative float'
        self.rel_tol = rel_tol
        self.vc_lb = np.ones(self.N)*np.inf
        self.x_ref = {}
        for var_pointer in self.vars_:
            self.x_ref[var_pointer] = np.zeros(var_pointer.shape)
            
        self.c_bound = c_bound
            
        self.cstr_gen = cstr_gen
        self.all_cstrs = [self.cstr_gen(scenario)-self.slack_feas[0]<=0 for scenario in self.scenarios]
        self.main_solver = used_solver
        self.individual_nus_comp = individual_nus_comp
        
    ### methods
        
    ##### returns (cuts-updated) feasible set considering a subset S of constraints
    def X_S(self,S=[],include_cuts=False):
        if include_cuts:
            return self.constraints+self.cuts+[self.all_cstrs[int(s)] for s in S]
        else:
            return self.constraints+[self.all_cstrs[int(s)] for s in S]
    
    ##### updates the indices yet to save or discard; i.e. to explore
    def _refresh(self):
        self.selectable = np.setdiff1d(np.setdiff1d(np.arange(self.N),self.safe_indices),self.pruned_indices)
        
    #### checks whether the probabilistic constraint is satisfied
    #### at the current value(s) assigned to the cvxpy Variable(s) if vec is None
    #### given the constraints values if vec is provided
    def _feasible(self,vec=None):
        if vec is None:
            return np.sum(self.pis[np.where(self.vc_lb<=TOLFEAS)[0]]) >= self.varsigma_prob
        elif isinstance(vec,np.ndarray) and len(vec)==self.N:
            return np.sum(self.pis[np.where(vec<=TOLFEAS)[0]]) >= self.varsigma_prob
        else:
            print('format error... check inputs')
            return False
        
    #### returns whether or not, based on the current knowledge
    #### represented by safe_indices and pruned_indices, if query belongs to every possible S in T(oplus,ominus) convex hull made 
    ####  xis from scenarios
    def _separation_scan(self,query,tol_diff=5e-6,angle_sep_threshold=1e-2,debug=False):

        ### check for duplicates of query in dataset ###
        locally_selectable = list(np.setdiff1d(np.arange(self.N), self.pruned_indices))
        N_local = len(locally_selectable)
        diff_norms = np.sqrt(np.sum((self.scenarios[locally_selectable]-np.outer(np.ones(N_local),query))**2,1))
        non_query_atoms_indices = [locally_selectable[int(ids)] for ids in np.where(diff_norms>tol_diff)[0]]
        non_query_atoms = self.scenarios[non_query_atoms_indices]
        N_effective = len(non_query_atoms)
        
        if np.sum(self.pis[non_query_atoms_indices]) < self.varsigma_prob:
            # -> no subset can be created with max_size atoms different from query <- #
            return True
        
        shifted_non_query_atoms = non_query_atoms-np.outer(np.ones(N_effective),query)

        ### dedicated approaches for lower-dimensional settings ###
                
        if self.p==2:
            
            # /!\ enhanced AngleScan procedure

            # Compute angles of non-query atoms relative to the query point
            angles = []
            angles_req = []
            for nqa_id,snqa in zip(non_query_atoms_indices,shifted_non_query_atoms):
                angle = math.atan2(snqa[1], snqa[0])
                angles.append(angle)
                if nqa_id in self.safe_indices:
                    angles_req.append(angle)
                
            if len(self.safe_indices)>0:
                angles_req = np.array(angles_req)
                tre_low,tre_high = min(angles_req),max(angles_req)
                if len(angles_req[angles_req>0])>0:
                    tre_low_bis = min(angles_req[angles_req>0])
                else:
                    tre_low_bis = math.pi # no cstr
                if len(angles_req[angles_req<0])>0:
                    tre_high_bis = max(angles_req[angles_req<0])+2*math.pi
                else:
                    tre_high_bis = math.pi # no cstr

            # Sort angles and handle the circular case with extended angles
            new_pis = self.pis[non_query_atoms_indices][np.argsort(angles)]
            new_pis = np.concatenate((new_pis,new_pis))
            angles.sort()
            extended_angles = angles + [a + 2 * math.pi for a in angles]
            max_weight,max_size = 0,self.varsigma_prob
            right = 0

            for left in range(N_effective):
                ref_ = extended_angles[left]
                target = ref_ + math.pi
                # Extend the window as far as possible
                # + angle_sep_threshold if we want to be sure not to select wrong indices ; - angle_sep_threshold otherwise
                while right < 2 * N_effective and extended_angles[right] <= target+angle_sep_threshold: 
                    right += 1
                current_weight = np.sum(new_pis[left:right+1])
                if current_weight > max_weight:
                    if len(self.safe_indices)>0:
                        if ref_ >= -math.pi and ref_<0: # 1st and 2nd quads
                            if ref_<=tre_low and target>=tre_high:
                                max_weight = current_weight
                        else: # 3rd and 4th quads
                            if ref_<=tre_low_bis and target>=tre_high_bis:
                                max_weight = current_weight
                    else:
                        max_weight = current_weight

                if max_weight>=max_size: # accelerate return
                    return False

            return True
        
        elif self.p==3:
            
            max_weight,max_size = 0,self.varsigma_prob
            
            # /!\ FacetScan procedure
            for s1,snqa_1 in enumerate(shifted_non_query_atoms):
                for s2,snqa_2 in enumerate(shifted_non_query_atoms):
                    if s1<s2:
                        coefs = np.array([snqa_1[1]*snqa_2[2]-snqa_1[2]*snqa_2[1],\
                                          snqa_1[2]*snqa_2[0]-snqa_1[0]*snqa_2[2],\
                                          snqa_1[0]*snqa_2[1]-snqa_1[1]*snqa_2[0]])
                        if np.linalg.norm(coefs)>tol_diff:
                            buf_vals = shifted_non_query_atoms@coefs
                            to_check1 = [non_query_atoms_indices[int(elem)] for elem in np.where(buf_vals>=0)[0]]
                            to_check2 = [non_query_atoms_indices[int(elem)] for elem in np.where(buf_vals<=0)[0]]
                            max_weight = max(np.sum(pis[to_check1]),np.sum(pis[to_check2]))
                        else:
                            if debug:
                                print('colinearity detected...')
                            max_weight = max_size
                        if max_weight>=max_size: # accelerate return
                            return False
            return True
        else:  
            return False # not implemented yet
        
    #### checks (and possibly adds) among selectable scenarios which belong to the cvx-hull of 'safed' indices;
    #### these indices should be part of the set safe_indices as well 
    def _chull_ref(self,tol_sel=5e-4): 

        cnt_ = 0

        if len(self.selectable)==0:
            return cnt_

        ## cvx hull refinement
        if len(self.safe_indices)>3 :
            try:
                cnt_ = 0
                ch = ConvexHull(points=self.scenarios[self.safe_indices])
                BETA,GAMMA = ch.equations[:,:-1],ch.equations[:,-1]
                for s,xi_s in zip(self.selectable,self.scenarios[self.selectable]):
                    if np.max(BETA@xi_s+GAMMA)<=tol_sel:
                        self.safe_indices.append(s)
                        cnt_ += 1
            except:
                print('-> numerical issue when dealing with cvx_hull equations... remaining of the step skipped.')
        return cnt_
    
    #### fixed knowledge
    def _compute_bigM(self,S,include_cuts=False,verb_bigM=False):
        if self.c_bound is None:
            return np.inf*np.ones(len(S))
        else:
            bigMs = [-np.inf]*len(S)
            X_fixed = self.X_S(self.safe_indices,include_cuts)
            xi_param_pointer = self.c_bound[0].parameters()[0]
            num_numerissues = 0
            if verb_bigM:
                print(' ')
            for idobj,obj_at_max in enumerate(self.c_bound):
                if verb_bigM:
                    print('bigM defining problems #'+str(idobj+1)+' over '+str(len(self.c_bound)))
                    print(' ')
                prob = cp.Problem(cp.Maximize(obj_at_max),constraints=X_fixed+[self.slack_feas[0]==0])
                for ids,s,xi_s in zip(np.arange(len(S)),S,self.scenarios[S]):
                    xi_param_pointer.value = xi_s
                    try:
                        if self.main_solver is None:
                            prob.solve(solver=cp.SCIPY,warm_start=True,scipy_options={'method':'highs'})
                        else:
                            prob.solve(solver=self.main_solver,warm_start=True)
                    except:
                        num_numerissues += 1
                    if verb_bigM and ids%int(max(1,len(S)/10))==0:
                        print('refined bigM of @'+str(s)+' | remaining computations [%] -> '+str((len(S)-(ids+1))/len(S)*100)[:5])
                    if prob.status=='optimal':
                        bigMs[ids] = max(bigMs[ids],self.all_cstrs[int(s)].expr.value)
                    else:
                        bigMs[ids] = np.inf
                if verb_bigM:
                    print(' ')
            if verb_bigM:
                if num_numerissues>0:
                    print('#numerical_issues: '+str(num_numerissues))
                    print(' ')
            return np.array(bigMs)
                        
        
    ###----------------------------------------------------------------------------------------
    
    ''' -> prune s based on achievable UB conditional to hypothesis S^* is a supset of S_safe U {s}'''
    def prune(self,include_cuts=False,verbose=True):
        
        num_pruned = 0
        prunable = np.setdiff1d(self.selectable,list(np.where(self.vc_lb<=TOLFEAS)[0]))
        
        if verbose:
            print(' ')
            
        if self.UB<np.inf:

            for ids,s in enumerate(prunable):

                sup_S = np.union1d(self.safe_indices,[s])
                prob_cut = cp.Problem(cp.Minimize(self.objective),self.X_S(sup_S,include_cuts))
                try:
                    if self.main_solver is None:
                        prob_cut.solve(solver=cp.SCIPY,warm_start=True,scipy_options={'method':'highs'})
                    else:
                        prob_cut.solve(solver=self.main_solver,warm_start=True)
                    slack_UB = prob_cut.value-self.UB

                    if verbose and ids%int(max(1,len(prunable)/10))==0:
                        print('trial @'+str(s)+' | remaining checks [%] -> '+str((len(prunable)-(ids+1))/len(prunable)*100)[:5])

                    if slack_UB>TOLPRUNE:
                        self.pruned_indices.append(s)
                        num_pruned += 1

                        if verbose:
                            print('pruned! @'+str(s))

                    elif slack_UB<0:
                        local_vc = np.array([self.all_cstrs[int(s)].expr.value for s in range(self.N)])
                        if self._feasible(vec=local_vc):
                            self.UB = prob_cut.value
                            self.es_ub = np.sum(self.pis[np.where(local_vc<=TOLFEAS)[0]])
                            for var_pointer in self.vars_:
                                self.x_ref[var_pointer] = var_pointer.value
                except:
                    if verbose:
                        print('numerical issue for @'+str(s)+'... hence not discarded')
                    pass
                        
            self._refresh()
            
        if verbose:
            print(' ')
            print('after optimality scan: '+str(num_pruned)+' indices were added to the pruned set')
            print(' ')
    
        return num_pruned
    
        
    '''-> certify that s belongs to oplus based on impossibility of the counterpart'''
    def certify(self,verbose=False,debug=False,max_time_cert=10.0):
        num_safed = 0
        
        time_start = time.time()
        
        for s,xi_s in zip(self.selectable,self.scenarios[self.selectable]):
            if self._separation_scan(query=xi_s,debug=debug):
                num_safed += 1
                self.safe_indices.append(s)
            if time.time()-time_start>max_time_cert:
                if verbose:
                    print('after separation scan: '+str(num_safed)+' indices were added to the safe set')
                self._refresh()
                break;
                
        if verbose:
            print('after separation scan: '+str(num_safed)+' indices were added to the safe set')
                
        self._refresh()
        
        ## cvx hull refinement
        cvx_hull_add = self._chull_ref()
        num_safed += cvx_hull_add
        
        if verbose:
            print('and after cvx-hull refinement: '+str(cvx_hull_add)+' more indices were added to the safe set')
        
        self._refresh()
            
        return num_safed 
      
    def update_LB(self,include_cuts=False,verb=False,verb_debug=False):
        prob_lb = cp.Problem(cp.Minimize(self.objective),self.X_S(self.safe_indices,include_cuts))
        if self.main_solver is None:
            try:
                prob_lb.solve(solver=cp.SCIPY,warm_start=True,scipy_options={'method':'highs'},verbose=verb_debug)
            except: 
                prob_lb.solve(solver=cp.MOSEK,warm_start=True,verbose=verb_debug)
        else:
            prob_lb.solve(solver=self.main_solver,warm_start=True,verbose=verb_debug)
        self.LB = max(prob_lb.value,self.LB)
        self.vc_lb = np.array([self.all_cstrs[int(s)].expr.value for s in range(self.N)])
        if verb:
            self.es_lb = np.sum(self.pis[np.where(self.vc_lb<=TOLFEAS)[0]])
            print(' ')
            print('empirical cstr (LB): '+str(self.es_lb))
            print('current lower-bound: '+str(self.LB))
            print(' ')
    
    def improve_UB(self,include_cuts=False,verb=False):
        
        ## compute the violations for each constraint at current x_lb (e.g. minimizer linked to LB)
        cvals = np.maximum(0,self.vc_lb)
        active_non_certified = list(np.where(cvals==0)[0])
        S_heuristic = list(np.union1d(self.safe_indices,active_non_certified))
        possible_set = np.setdiff1d(self.selectable,S_heuristic)

        ## keep cvals for indices yet to select
        cvals_possible = cvals[possible_set]
        rank = np.argsort(cvals_possible)

        lower_size = 0
        upper_size = len(possible_set)
                        
        counter_safeguard,num_safeguard = 0,100
        while upper_size-lower_size>1 and counter_safeguard<num_safeguard:
            counter_safeguard+=1
            target_size = int(np.ceil((upper_size+lower_size)/2))
            elected = rank[:target_size]
            expanded_set = list(np.union1d(S_heuristic,possible_set[elected]))
            prob_ub = cp.Problem(cp.Minimize(self.objective),self.X_S(expanded_set,include_cuts))
            try:
                if self.main_solver is None:
                    prob_ub.solve(solver=cp.SCIPY,warm_start=True,scipy_options={'method':'highs'})
                else:
                    prob_ub.solve(solver=self.main_solver,warm_start=True)
                vec_ub = np.array([self.all_cstrs[int(s)].expr.value for s in range(self.N)])
                es = np.sum(self.pis[np.where(vec_ub<=TOLFEAS)[0]])
            except:
                if verb:
                    print('numerical issue or problem might be infeasible... we continue the local-scanning')
                es = 0.0
            if es>=self.varsigma_prob:
                pub = prob_ub.value
                if pub<self.UB:
                    self.es_ub = es
                    self.UB = pub
                    for var_pointer in self.vars_:
                        self.x_ref[var_pointer] = var_pointer.value
                upper_size = target_size
            else:
                lower_size = target_size
        
        if verb:
            print(' ')
            print('empirical cstr (UB): '+str(self.es_ub))
            print('current upper-bound: '+str(self.UB))
            print(' ')
            
    def _rel_optimality(self):
        if self.UB==np.inf or self.LB==-np.inf:
            return np.inf
        return (self.UB-self.LB)/max(1,(abs(self.UB)+abs(self.LB))/2)

    def pre_solve(self,include_cuts=False,verbose_general=False,T_max=1200,\
                  verbose_certify=False,verbose_prune=False,display=False,max_time_cert=10,max_rounds=1, *args, **kwargs):
        
        last_rel = np.inf
        accumulation_counter = 0 
        
        time_ref = time.time()
        k_cert,k_lb,k_ub,k_prune = 0,0,0,0
        
        ## pre-phase
        if self.individual_nus_comp is not None:
            print(' ')
            print('pre-pass oracle...')
            lb_trial = np.inf
            individual_nus = np.inf*np.ones(N)
            self.slack_feas.value = np.zeros(1)
            for s in range(self.N):
                nu_s,x_s_list = self.individual_nus_comp(self.scenarios[int(s)])
                individual_nus[s] = nu_s
                for var_pointer,x_s in zip(self.vars_,x_s_list):
                    var_pointer.value = x_s
                lb_trial = min(lb_trial,nu_s)
                loc_vc_s = np.array([self.all_cstrs[int(s)].expr.value for s in range(self.N)])
                es = np.sum(self.pis[np.where(loc_vc_s<=TOLFEAS)[0]])
                if es>=self.varsigma_prob:
                    if self.UB>nu_s:
                        self.UB = nu_s
                        self.es_ub = es
                        for var_pointer,x_s in zip(self.vars_,x_s_list):
                            self.x_ref[var_pointer] = x_s
            self.LB = lb_trial
            self.pruned_indices += list(np.where(individual_nus>self.UB)[0])
            self._refresh()
            print('LB = '+str(self.LB)+' | self.UB = '+str(self.UB)+' | pre-pruned: '+str(len(self.pruned_indices)))
            print(' ')

        n_rounds = 0
        while time.time()-time_ref<=T_max and n_rounds<max_rounds:
            
            ## first phase
            
            self.certify(include_cuts,verbose_general and verbose_certify,max_time_cert)
            try:
                self.update_LB(verb=verbose_general)
            except:
                print(' ')
                print('=> instability while solving relaxation....')
                pass
            if self.slack_feas.value[0]>1e-1:
                print(' ')
                print('total running time: '+str(time.time()-time_ref)+' [s]')
                print('numerical issue or problem infeasible.... check problem data')
                return 'infeasible',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
            
            k_cert+=1
            k_lb+=1
            
            if self._feasible():
                
                if self.LB<self.UB: 
                    for var_pointer in self.vars_:
                        self.x_ref[var_pointer] = var_pointer.value
                    self.UB = self.LB
                    
            if display:
                showcase(self.scenarios,safe_indices=self.safe_indices,pruned_indices=self.pruned_indices)
            
            local_rel_tol = self._rel_optimality()
            if local_rel_tol>=last_rel:
                accumulation_counter+=1
            last_rel = local_rel_tol
            if local_rel_tol<=self.rel_tol:
                
                if verbose_general:
                    print(' ')
                    print('total running time: '+str(time.time()-time_ref)+' [s]')
                    print('target accuracy reached:: '+str(local_rel_tol))  
                    print('cost:: '+str(self.UB))
                
                return 'optimal',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
            
            ## second phase
                
            self.improve_UB(verb=verbose_general)
            k_ub+=1
            
            local_rel_tol = self._rel_optimality()
            if local_rel_tol>=last_rel:
                accumulation_counter+=1
            last_rel = local_rel_tol
            if local_rel_tol<=self.rel_tol:
                
                if verbose_general:
                    print(' ')
                    print('total running time: '+str(time.time()-time_ref)+' [s]')
                    print('target accuracy reached:: '+str(local_rel_tol))  
                    print('cost:: '+str(self.UB))
                if display:
                    for vari in self.vars_:
                        vari.value = self.x_ref[vari]
                    local_vc = np.array([self.all_cstrs[int(s)].expr.value for s in range(self.N)])
                    showcase(self.scenarios,safe_indices=self.safe_indices,pruned_indices=self.pruned_indices,\
                             extra_selected_indices=np.setdiff1d(list(np.where(local_vc<=TOLFEAS)[0]),self.safe_indices))
                
                return 'optimal',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
            
            self.prune(include_cuts,verbose_general and verbose_prune)
            k_prune+=1
            
            ## third phase (cuts) -> not working in a stable manner right now...
            if include_cuts:
                self.cuts = [self.objective<=self.UB]
                
            ## last phase (accumulation...)
            if accumulation_counter>=3:
                if verbose_general:
                    print(' ')
                    print(' /!\ pre_solve termination /!\ ')
                    print('safe indices: '+str(len(self.safe_indices))+' ; pruned indices: '+str(len(self.pruned_indices)))
                return 'optimal_inaccurate',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
            
            
            n_rounds += 1

            
            if display:
                showcase(self.scenarios,safe_indices=self.safe_indices,pruned_indices=self.pruned_indices)
    
        if verbose_general:
            print(' ')
            print(' /!\ pre_solve termination /!\ ')
            print('safe indices: '+str(len(self.safe_indices))+' ; pruned indices: '+str(len(self.pruned_indices)))
            print('best feasible solution:: '+str(self.UB))
        if self.UB<np.inf:
            return 'optimal_inaccurate',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
        elif self.LB==-np.inf:
            return 'unbounded',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
        else:
            return 'infeasible',time.time()-time_ref,k_cert,k_lb,k_ub,k_prune
        
        
    def solve(self,include_cuts=False,verbose_solver=True,T_max=1200,\
              skip_presolve=False,*args,**kwargs):
        
        '''assert unconstrained feasibility'''
        t0 = time.time()
        try:
            prob_unc = cp.Problem(cp.Minimize(self.objective),self.X_S([]))
            if self.main_solver is None:
                prob_unc.solve(solver=cp.SCIPY,scipy_options={'method':'highs'})
            else:
                prob_unc.solve(solver=self.main_solver)
        except:
            print('trying another solver...')
            self.main_solver = cp.MOSEK
            try:
                prob_unc.solve(solver=self.main_solver)
                if prob_unc.status=='optimal' or prob_unc.status=='unbounded':
                    self.LB = prob_unc.value
                else:
                    print(' ')
                    print('total running time: '+str(time.time()-t0)+' [s]')
                    print('numerical issue or problem infeasible.... check problem data')
                    return 'infeasible',time.time()-t0,0,0,0,0
            except:
                print(' ')
                print('total running time: '+str(time.time()-t0)+' [s]')
                print('numerical issue or problem infeasible.... check problem data')
                return 'infeasible',time.time()-t0,0,0,0,0
        t0_elapsed = time.time()-t0
        
        if skip_presolve:
            status,timing,k_cert_,k_lb_,k_ub_,k_prune_ = None,0,0,0,0,0
        else:
            status,timing,k_cert_,k_lb_,k_ub_,k_prune_ = self.pre_solve(**kwargs) 
            if status=='optimal' or status=='infeasible' or status=='unbounded':
                return status,timing,k_cert_,k_lb_,k_ub_,k_prune_
        
        time_ref2 = time.time()
        
        ddof = self.varsigma_prob-np.sum(self.pis[self.safe_indices])
        t_select = cp.Variable(len(self.selectable),boolean=True)
        local_bigMs = self._compute_bigM(self.selectable,include_cuts,verb_bigM=False)
        
        cstrs_select = [self.pis[self.selectable]@t_select>=ddof]
            
        new_X = self.X_S(self.safe_indices,include_cuts)
        final_X = new_X+cstrs_select
        for ids,s in enumerate(self.selectable):
            final_X += [self.cstr_gen(self.scenarios[int(s)])-self.slack_feas[0]<=(1.0-t_select[ids])*local_bigMs[ids]]
        final_prob = cp.Problem(cp.Minimize(self.objective),final_X)
        
        final_prob.solve(solver=cp.MOSEK,verbose=verbose_solver,mosek_params = {'MSK_DPAR_OPTIMIZER_MAX_TIME': max(10,T_max-(timing+(time.time()-time_ref2)+t0_elapsed))})

        TF = time.time()-time_ref2+timing+t0_elapsed
        if final_prob.status=='optimal':
            self.UB = final_prob.value
            self.LB = final_prob.value
            for var_pointer in self.vars_:
                self.x_ref[var_pointer] = var_pointer.value
            
            print('final target accuracy:: 0.0')
            print('total running time: '+str(TF)+' [s]')
            print('cost:: '+str(self.UB))
            return 'optimal',TF,k_cert_,k_lb_,k_ub_,k_prune_
        else:
            self.UB = min(self.UB,final_prob.value)
            print('final target accuracy:: '+str(self._rel_optimality()))
            print('total running time: '+str(TF)+' [s]')
            print('cost:: '+str(self.UB))
            return 'optimal_inaccurate',TF,k_cert_,k_lb_,k_ub_,k_prune_

In [ ]:
def showcase(xi_list,safe_indices=[],pruned_indices=[],extra_selected_indices=None):
    fig, ax = plt.subplots(figsize=(6,6))
    plt.grid()
    plt.xlabel('$\\xi_1$')
    plt.ylabel('$\\xi_2$')
    plt.title('data points')
    plt.scatter(xi_list[:,0],xi_list[:,1],label='data points')
    if extra_selected_indices is not None:
        plt.scatter(xi_list[extra_selected_indices,0],xi_list[extra_selected_indices,1],label='selected @x_return',color='orange')
    plt.scatter(xi_list[safe_indices,0],xi_list[safe_indices,1],label='safely selected',color='green')
    plt.scatter(xi_list[pruned_indices,0],xi_list[pruned_indices,1],label='pruned',color='red')
    plt.legend();